In [ ]:
# Page 129 of Python for Algorithmic Trading from idea to cloud deployment 
# Machine Learning
# Linear Regression
# Predicting price od EURUSD

In [ ]:
import pandas as pd
import os 
import random 
import numpy as np 
from pylab import mpl, plt 

plt.style.use('seaborn') 
mpl.rcParams['savefig.dpi'] = 300 
mpl.rcParams['font.family'] = 'serif' 
os.environ['PYTHONHASHSEED'] = '0'


In [ ]:
raw = pd.read_csv('http://hilpisch.com/pyalgo_eikon_eod_data.csv', index_col=0, parse_dates=True).dropna()
#raw.info()

# Same Procedure as in the example before this but lag taken as 5 instead of 3
#  We are trying to predict PRICE LEVEL itself for the next day based on Today's price. 

In [365]:
symbol = 'EUR='
data = pd.DataFrame(raw[symbol])  # extract EURUSD from the df
data.rename(columns={symbol: 'price'}, inplace=True)
train_data=pd.DataFrame()
train_data['price'] = data['price'].loc[:'2019-10-31']
#print(data.loc['2019-11-04'])
test_data=pd.DataFrame()
test_data['price'] = data['price'].loc['2019-11-1':]
train_data

price
Date              
2010-01-04  1.4411
2010-01-05  1.4368
2010-01-06  1.4412
2010-01-07  1.4318
2010-01-08  1.4412
...            ...
2019-10-25  1.1078
2019-10-28  1.1098
2019-10-29  1.1110
2019-10-30  1.1148
2019-10-31  1.1150

[2475 rows x 1 columns]

In [366]:
def LR_Test(lags, data1):
    #lags=5
    data=pd.DataFrame(index=data1.index)
    data['price']=data1['price']
    cols = []
    #print(data)
    for lag in range(1, lags + 1):                                           #This loop will delete first 5 rows on every call of function
        col = f'lag_{lag}' 
        data[col] = data['price'].shift(lag) 
        cols.append(col) 
    #print(data)    
    data.dropna(inplace=True)  #-------------------------
    #print(data)
    #print(data[cols])
    reg = np.linalg.lstsq(data[cols], data['price'], rcond=None)[0]
    #print(reg)
    data['prediction'] = np.dot(data[cols], reg)
    #data[['price', 'prediction']].plot(figsize=(10, 6));
    #data[['price', 'prediction']].loc['2019-10-1':].plot( figsize=(10, 6));
    #print(data)
    return data

In [ ]:
#reg = np.linalg.lstsq(data[cols], data['price'], rcond=None)[0]  #Here, data[cols] is the independent variables & price is dependent variable 
#reg # As we can see from results of regression, 98.64% is explained by the yesterday's price only and that's COOOOOOLLLL

In [ ]:
#data['prediction'] = np.dot(data[cols], reg)
#data[['price', 'prediction']].plot(figsize=(10, 6));
#data[['price', 'prediction']].loc['2019-12-1':].plot( figsize=(10, 6)); # We decrease the time to get an idea



In [367]:
#print(train_data)
work_data=pd.DataFrame()#data=None, columns=test_data.columns, index=test_data.index)
work_data = train_data.copy()

#Compare and store the prediction with the unseen data & than add the same to train_data
# Means add one row and retrain the data to get new prediction for next day & continue the process till
#  test data is exhausted 

res_elva_test = []
for i in range(0,test_data.shape[0]) :
    wo_data=pd.DataFrame()#data=None, columns=test_data.columns)#, index=test_data.index)
    work_data = LR_Test(3,work_data)                             # I am using 1 lag instead of 5 lags -----------------------------
    #print(work_data)
    #print(work_data['price'][-2])
    #print(work_data['prediction'][-2])
    #print(work_data['price'][-1])
    Test_List = [round(work_data['price'][-2],5), round(work_data['prediction'][-2],5), round(work_data['price'][-1],5)]
    Pred_Direc = -1 if ((Test_List[1] - Test_List[0]) < 0 ) else 1
    Test_List.append(Pred_Direc)
    Actual_Direc = -1 if ((Test_List[2] - Test_List[0]) < 0 ) else 1
    Test_List.append(Actual_Direc)
    res_elva_test.append(Test_List)
    #print(Test_List)
    entries=[]
    entries.append((test_data.iloc[[i]]))
    wo_data = pd.concat(entries)
   
    work_data = pd.concat([work_data, wo_data])


print(res_elva_test)

[[1.1148, 1.11086, 1.115, -1, 1], [1.115, 1.11461, 1.1165, -1, 1], [1.1165, 1.11489, 1.1126, -1, -1], [1.1126, 1.11634, 1.1074, 1, -1], [1.1074, 1.11255, 1.1065, 1, -1], [1.1065, 1.10733, 1.1049, 1, -1], [1.1049, 1.10634, 1.1016, 1, -1], [1.1016, 1.10481, 1.1032, 1, 1], [1.1032, 1.10153, 1.1007, -1, -1], [1.1007, 1.10301, 1.1006, 1, -1], [1.1006, 1.10065, 1.1021, 1, 1], [1.1021, 1.10047, 1.105, -1, 1], [1.105, 1.10197, 1.107, -1, 1], [1.107, 1.10486, 1.1078, -1, 1], [1.1078, 1.10689, 1.1072, -1, -1], [1.1072, 1.10769, 1.1057, 1, -1], [1.1057, 1.1071, 1.1022, 1, -1], [1.1022, 1.1056, 1.1013, 1, -1], [1.1013, 1.10214, 1.1018, 1, 1], [1.1018, 1.10116, 1.0998, -1, -1], [1.0998, 1.10167, 1.1015, 1, 1], [1.1015, 1.09974, 1.1077, -1, 1], [1.1077, 1.10133, 1.1081, -1, 1], [1.1081, 1.10751, 1.1076, -1, -1], [1.1076, 1.10809, 1.1102, 1, 1], [1.1102, 1.10751, 1.1057, -1, -1], [1.1057, 1.11005, 1.1062, 1, 1], [1.1062, 1.10575, 1.1092, -1, 1], [1.1092, 1.10604, 1.1128, -1, 1], [1.1128, 1.10907, 1.1

In [368]:
P=0
N=0
for i in range(0,len(res_elva_test)):
    if res_elva_test[i][3]==res_elva_test[i][4]:
        P+=1
        
    else:
        N+=1
print(P, N)
print(f"Accuracy = %s Percent"% round((P/(P+N))*100,2))

18 23
Accuracy = 43.9 Percent


In [ ]:
work_data[0]['price']#.show(view='Categorical')

In [ ]:
x=res_elva_test[res_elva_test[[3]]==res_elva_test[[4]]
print(x)